In [1]:
import requests
import pprint
import pandas as pd
import numpy as np
import json
from chicken_dinner.models.tournament import Tournament
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("API_KEY")

In [2]:
# tournament_list.json 파일 불러오기
file_path = "./tournament_list.json"

with open(file_path, 'r') as file:
    tournament_id_dict = json.load(file)

In [3]:
header = {
  "Authorization": api_key,
  "Accept": "application/vnd.api+json"
}

# tournament_index를 지정하여 원하는 위치의 id값 출력
tournament_index = 10
tournament_id = tournament_id_dict['data'][tournament_index]['id']

# 해당 tournament_id를 불러오기 위한 link 작성
url_temp = "https://api.pubg.com/tournaments/"
url = url_temp + tournament_id

# requests.get 함수를 이용하여 tournament_list를 json으로 호출
r = requests.get(url, headers=header)
current_tournament = r.json()

# 모든 match_id와 created_at을 dict으로 출력
matchId_dict = {match['attributes']['createdAt']: match['id'] for match in current_tournament['included']}

# matchId_dict를 createdAt로 정렬 후 dataframe으로 변환
cur_tournament_matchId_df = pd.DataFrame(sorted(matchId_dict.items(), key=lambda x: x[0]), columns=['createdAt', 'matchId'])

tournament_id

'as-pp1wf4'

In [4]:
from chicken_dinner.pubgapi import PUBG

# PUBG class를 토너먼트용 class로 custom
PUBG = PUBG(api_key=api_key, shard='pc-tournament', gzip=True)

# match_participant
player_id = []
team_roster_id = []
team_id = []
team_rank = []
match_id = []
participant_stats = []

# 
for i in cur_tournament_matchId_df['matchId']:
  match = PUBG.match(i)
  rosters = match.rosters
  for i in range(len(rosters)):
    roster = rosters[i]
    roster_participant = roster.participants
    for i in range(len(roster_participant)):
      participant = roster_participant[i]
      match_id.append(match.id)
      player_id.append(participant.name)
      team_roster_id.append(roster.id)
      team_rank.append(roster.stats['rank'])
      team_id.append(roster.stats['team_id'])
      stats = participant.stats
      participant_stats.append(stats)

# Dataframe 생성
match_participant = pd.DataFrame({'match_id': match_id, 'player_id': player_id, 'team_roster_id': team_roster_id, 'team_id': team_id, 'team_rank': team_rank})
match_participant_stats = pd.DataFrame(participant_stats).drop(columns='player_id')

# 인덱스 기준으로 join
match_participant_all = pd.merge(match_participant, match_participant_stats, how='inner', left_index=True, right_index=True) 

# Dataframe 확인
match_participant_all

,match_id,player_id,team_roster_id,team_id,team_rank,dbnos,assists,boosts,damage_dealt,death_type,...,revives,ride_distance,road_kills,swim_distance,team_kills,time_survived,vehicle_destroys,walk_distance,weapons_acquired,win_place
0,6e9215d0-9352-4049-84fd-e02e4fe9f3a5,MG_SeungYong,30dcc6d4-a390-4e8d-84ef-a71b10fe47d9,2,14,1,0,2,78.489105,byplayer,...,0,5881.8706,0,0.0,0,1116,0,579.06036,3,14
1,6e9215d0-9352-4049-84fd-e02e4fe9f3a5,MG_Dlaks,30dcc6d4-a390-4e8d-84ef-a71b10fe47d9,2,14,1,1,4,341.791080,byplayer,...,2,5840.1133,0,0.0,0,1116,0,1303.39920,6,14
2,6e9215d0-9352-4049-84fd-e02e4fe9f3a5,MG_yacha,30dcc6d4-a390-4e8d-84ef-a71b10fe47d9,2,14,1,1,2,85.162150,byplayer,...,0,5310.6610,0,0.0,0,1116,0,710.62620,3,14
3,6e9215d0-9352-4049-84fd-e02e4fe9f3a5,MG_Gak,30dcc6d4-a390-4e8d-84ef-a71b10fe47d9,2,14,1,1,4,159.442370,byplayer,...,0,6191.0044,0,0.0,0,1099,0,811.58350,3,14
4,6e9215d0-9352-4049-84fd-e02e4fe9f3a5,EOS_KimKaXhaxx,b0fdddff-1b7f-4ad2-b9de-559759c73bf7,7,12,1,0,4,112.711150,byplayer,...,1,6150.1265,0,0.0,0,1146,0,1178.75320,9,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,03297124-f74f-4a6b-8eb7-45655b3231cd,EOS_Da-min,7c3f49a1-1833-430d-bec8-d203af1fc02d,7,12,0,1,1,0.000000,byplayer,...,0,3932.4836,0,0.0,0,854,0,564.46660,5,12
636,03297124-f74f-4a6b-8eb7-45655b3231cd,USG_Americano,9f69dc4b-e8e6-4215-8852-6739d120d704,9,3,4,0,8,675.326840,byzone,...,0,2936.1320,0,0.0,0,1652,0,2191.98410,4,3
637,03297124-f74f-4a6b-8eb7-45655b3231cd,USG_SHEVA,9f69dc4b-e8e6-4215-8852-6739d120d704,9,3,0,0,0,0.000000,byplayer,...,0,2126.9940,0,0.0,0,402,0,443.57816,5,3
638,03297124-f74f-4a6b-8eb7-45655b3231cd,USG_Kein,9f69dc4b-e8e6-4215-8852-6739d120d704,9,3,1,2,7,417.439500,byplayer,...,0,2453.4468,0,0.0,0,1652,0,1854.90970,7,3


In [5]:
# 불필요한 항목 제거
## DBNO : Down But Not Out
new_table = match_participant_all.drop(['match_id', 'death_type', 'team_roster_id', 'team_id', 'name', 'vehicle_destroys', 'weapons_acquired'], axis='columns')
new_table

,player_id,team_rank,dbnos,assists,boosts,damage_dealt,headshot_kills,heals,kill_place,kill_streaks,kills,longest_kill,revives,ride_distance,road_kills,swim_distance,team_kills,time_survived,walk_distance,win_place
0,MG_SeungYong,14,1,0,2,78.489105,0,2,33,1,1,6.204905,0,5881.8706,0,0.0,0,1116,579.06036,14
1,MG_Dlaks,14,1,1,4,341.791080,1,1,31,1,1,16.384058,2,5840.1133,0,0.0,0,1116,1303.39920,14
2,MG_yacha,14,1,1,2,85.162150,0,0,32,1,1,55.633865,0,5310.6610,0,0.0,0,1116,710.62620,14
3,MG_Gak,14,1,1,4,159.442370,1,0,34,1,1,25.437954,0,6191.0044,0,0.0,0,1099,811.58350,14
4,EOS_KimKaXhaxx,12,1,0,4,112.711150,0,3,30,1,1,5.279407,1,6150.1265,0,0.0,0,1146,1178.75320,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,EOS_Da-min,12,0,1,1,0.000000,0,1,52,0,0,0.000000,0,3932.4836,0,0.0,0,854,564.46660,12
636,USG_Americano,3,4,0,8,675.326840,2,4,1,3,7,92.984520,0,2936.1320,0,0.0,0,1652,2191.98410,3
637,USG_SHEVA,3,0,0,0,0.000000,0,0,37,0,0,0.000000,0,2126.9940,0,0.0,0,402,443.57816,3
638,USG_Kein,3,1,2,7,417.439500,0,2,15,1,1,43.633286,0,2453.4468,0,0.0,0,1652,1854.90970,3


In [6]:
new_table.keys()

Index(['player_id', 'team_rank', 'dbnos', 'assists', 'boosts', 'damage_dealt',
       'headshot_kills', 'heals', 'kill_place', 'kill_streaks', 'kills',
       'longest_kill', 'revives', 'ride_distance', 'road_kills',
       'swim_distance', 'team_kills', 'time_survived', 'walk_distance',
       'win_place'],
      dtype='object')